In [ ]:
import duckdb
import pandas as pd
from pathlib import Path

PARENT_FOLDER = Path.cwd().parent
DATA_FOLDER = PARENT_FOLDER / "data"

In [ ]:
# Initialize duckdb
conn = duckdb.connect(DATA_FOLDER / "main.duckdb")
conn.sql("CALL start_ui();")

## Ingestion

In [ ]:
q_ingestion = """--sql
    CREATE OR REPLACE TABLE bronze_scan_results AS
        SELECT
            md5("F  ile Path") AS "md5_hash_id",
            *
        FROM read_csv($file_path)
"""
conn.sql(query=q_ingestion, params={
    "file_path": str(DATA_FOLDER / "scan_result.csv")
})

conn.sql("""--sql
         SELECT *
         FROM bronze_scan_results
         """)

In [ ]:


def analyze():
    conn.sql("""--sql
        SELECT
                md5("File Path") AS "md5_hash_id",
                "Issues JSON"
        FROM read_csv(?)
        GROUP BY ALL
        HAVING MAX("Issue Count") = 5
    """, params=[str(DATA_FOLDER / "scan_result.csv")]).show()


if __name__ == "__main__":
    analyze()